In [1]:
from cobra.io import read_sbml_model

In [2]:
# ONLY  OUTCOMENT THESE LINES IF NEEDED AND COMMENT THEM AGAIN AFTERWARDS
# Verify working directory and change it if needed
#os.getcwd()
#os.chdir("/Users/abril/Documents/DTU/CFD_Resveratrol/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae")

In [4]:
model = read_sbml_model("../data/models/yeast8_resv_glc.xml")

In [5]:
# Model olverview
model

Name,yeastGEM_v8__46__6__46__2
Memory address,0x012270baf0
Number of metabolites,2747
Number of reactions,4067
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


Before doing any modifications to the biomass, glucose, resveratrol, or other fluxes, let's see what is the resveratrol yield an prodcution with the default boundaries provided byt the model.

In [28]:
with model:
    model.objective = model.reactions.VVVST1 # set the objective reaction to resveratrol production.
    resv_production = model.optimize().objective_value
    print("Maximum resveatrol productivity  [mmol/gDW^-1 h^-1] = ", resv_production)
    print("Theoretical max. yield [mmol-resv/mmol-glc ]:", resv_production / (-1*model.reactions.EX_glc__D_e.flux))

Maximum resveatrol productivity  [mmol/gDW^-1 h^-1] =  -7.284844458383337e-16
Theoretical max. yield [mmol-resv/mmol-glc ]: -7.284844458383337e-16


Now let's explore the model :)

In [11]:
# Verify if there is already an objective function (model exploration)
print(model.objective)

Maximize
1.0*GROWTH - 1.0*GROWTH_reverse_c845e


In [29]:
# The present objective function belongs to:
model.reactions.GROWTH

Reaction identifier,GROWTH
Name,growth
Memory address,0x123a33040
Stoichiometry,biomass_c --> biomass -->
GPR,
Lower bound,0.08192811254621667
Upper bound,1000.0


Now let's impose a minimum biomass bound. In this case 50% of the biomass flux.

In [17]:
biomass_objective = model.optimize().objective_value #get the objective value 

In [42]:
model.reactions.GROWTH.lower_bound = biomass_objective*0.5 #set the minimum biomass to be 50% of the flux
print("Biomass bounds nor are:", model.reactions.GROWTH.bounds)

Biomass bounds nor are: (0.04096405627310833, 1000.0)


## Maximum yield and productivity 

### Calculate theoretical maximum yields for chosen product for suitable carbon sources

In [43]:
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x124dffb20
Stoichiometry,e4_coumaroyl_coa + 3.0 h_c + 3.0 malcoa_c --> 4.0 co2_c + 4.0 coa_c + trans_resv + 3.0 H+ + 3.0 malonyl-CoA --> 4.0 carbon dioxide + 4.0 coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


##### Calculating maximum productivity and theoretical yield.
Using default medium concentration with minimum biomass set.

In [44]:
## Setting the objective reaction to resveratrol production.
with model:
    model.objective = model.reactions.VVVST1
    resv_production = model.optimize().objective_value
    print("Maximum resveatrol productivity  [mmol/gDW^-1 h^-1] = ", resv_production)
    print("Theoretical max. yield [mmol-resv/mmol-glc ]:", resv_production / (-1*model.reactions.EX_glc__D_e.flux))

Maximum resveatrol productivity  [mmol/gDW^-1 h^-1] =  0.15524702316800987
Theoretical max. yield [mmol-resv/mmol-glc ]: 0.15524702316800987


### Maximum yield and productivity increasing carbon source.

In [45]:
# Get the medium composition concentration based on the exchange reactions.
for m in model.medium.items():
    print(model.reactions.get_by_id(m[0]), "|", model.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

EX_nh4_e: nh4_e <=>  | ammonium exchange | conc. = 1000.0
EX_glc__D_e: glc__D_e <=>  | D-glucose exchange | conc. = 1.0
EX_h_e: h_e <=>  | H+ exchange | conc. = 1000.0
EX_fe2_e: fe2_e <=>  | iron(2+) exchange | conc. = 1000.0
EX_o2_e: o2_e <=>  | oxygen exchange | conc. = 1000.0
EX_pi_e: pi_e <=>  | phosphate exchange | conc. = 1000.0
EX_k_e: k_e <=>  | potassium exchange | conc. = 1000.0
EX_na1_e: na1_e <=>  | sodium exchange | conc. = 1000.0
EX_so4_e: so4_e <=>  | sulphate exchange | conc. = 1000.0
EX_h2o_e: h2o_e <=>  | water exchange | conc. = 1000.0
EX_cl_e: cl_e <=>  | chloride exchange | conc. = 1000.0
EX_cu2_e: cu2_e <=>  | Cu2(+) exchange | conc. = 1000.0
EX_mn2_e: mn2_e <=>  | Mn(2+) exchange | conc. = 1000.0
EX_zn2_e: zn2_e <=>  | Zn(2+) exchange | conc. = 1000.0
EX_mg2_e: mg2_e <=>  | Mg(2+) exchange | conc. = 1000.0
EX_ca2_e: ca2_e <=>  | Ca(2+) exchange | conc. = 1000.0


Inscreasing glucose concentration.

In [46]:
model.reactions.EX_glc__D_e.flux 

-1.0

In [54]:
# Growth rate with default glucose cocnentration
medium = model.medium
with model:
    growth_rate = model.optimize().objective_value  #Growth rate
    glc_con = (-1*model.reactions.EX_glc__D_e.flux)
    print("Glucose concentration is:", round(glc_con,3))
    print("Maximum growth rate is  [h^-1] = ", round(growth_rate,3), "\n")

    # Increase glucose concentration.
    medium['EX_glc__D_e'] = 10 #Glucose concentration to have the max growth rate.
    model.medium = medium

    #Re-calculate growth rate with new glucose concentration
    growth_rate = model.optimize().objective_value
    glc_con = (-1*model.reactions.EX_glc__D_e.flux)
    print("Glucose concentration is:", round(glc_con,3))
    print("Maximum growth rate is  [h^-1] = ",round(growth_rate,3), "\n")

        # Increase glucose concentration.
    medium['EX_glc__D_e'] = 30 #Glucose concentration to have the max growth rate.
    model.medium = medium

    #Re-calculate growth rate with new glucose concentration
    growth_rate = model.optimize().objective_value
    glc_con = (-1*model.reactions.EX_glc__D_e.flux)
    print("Glucose concentration is:", round(glc_con,3))
    print("Maximum growth rate is  [h^-1] = ",round(growth_rate,3))

Glucose concentration is: 1.0
Maximum growth rate is  [h^-1] =  0.082 

Glucose concentration is: 10.0
Maximum growth rate is  [h^-1] =  0.847 

Glucose concentration is: 30.0
Maximum growth rate is  [h^-1] =  2.548


We performed this simulation to know the limits of our system. According to the results, a glucose concentration of 1646.00 results in the maximum cell growth rate. However this does not mean it is the ideal glucose concentration. To make assumptions it it necessary to also observ the effects increasing the glucose concentration has on resveratrol theoretical productivity and yield.

##### Now lets see the effects of increasing glucose concentration on resveratrol yield and production.

In [59]:
medium = model.medium

with model:
    model.objective = model.reactions.VVVST1
    resv_production = model.optimize().objective_value
    print("With glucose concentration", (-1*model.reactions.EX_glc__D_e.flux))
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3), "\n")

    medium['EX_glc__D_e'] = 10
    model.medium = medium

    resv_production = model.optimize().objective_value
    print("With glucose concentration",(-1*model.reactions.EX_glc__D_e.flux))
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3),  "\n")

    medium['EX_glc__D_e'] = 30
    model.medium = medium

    resv_production = model.optimize().objective_value
    print("With glucose concentration",(-1*model.reactions.EX_glc__D_e.flux))
    print("Maximum resveatrol productivity  [mmol gDW^-1 h^-1] = ",round(resv_production,3))
    print("Theoretical max. yield [mmol-glc / mmol-resv]:", round(resv_production / (-1*model.reactions.EX_glc__D_e.flux),3))

With glucose concentration 1.0
Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  0.155
Theoretical max. yield [mmol-glc / mmol-resv]: 0.155 

With glucose concentration 10.0
Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  3.054
Theoretical max. yield [mmol-glc / mmol-resv]: 0.305 

With glucose concentration 30.0
Maximum resveatrol productivity  [mmol gDW^-1 h^-1] =  9.448
Theoretical max. yield [mmol-glc / mmol-resv]: 0.315


In [62]:
model.summary() 

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,1.778E-05,0,0.00%
cl_e,EX_cl_e,0.0001057,0,0.00%
cu2_e,EX_cu2_e,5.399E-05,0,0.00%
fe2_e,EX_fe2_e,2.573E-06,0,0.00%
glc__D_e,EX_glc__D_e,1,6,100.00%
h_e,EX_h_e,0.04144,0,0.00%
k_e,EX_k_e,0.0002974,0,0.00%
mg2_e,EX_mg2_e,0.0001018,0,0.00%
mn2_e,EX_mn2_e,0.0002237,0,0.00%
na1_e,EX_na1_e,0.0003253,0,0.00%
